# [Python] Crypto: REST API & candlestick graph

Hello :) !

In this exercise, we use the <a href="https://www.coingecko.com/en/api?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0101ENSkillsNetwork19487395-2022-01-01">CoinGecko API</a> to create interactive candlestick graphs for Bitcoin with Plotly.

I will use the API to get the price data for 90 days with 24 observation per day, 1 per hour. We will find the max, min, open, and close price per day meaning we will have 90 candlesticks and use that to generate the candlestick graph. We will add a 7-day moving average.

The Python client/wrapper for the API called <a href="https://github.com/man-c/pycoingecko?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0101ENSkillsNetwork19487395-2022-01-01">PyCoinGecko</a>  will make the requests easing to make.

## Table of contents:
* [Settings](#zero)
* [Data collection](#one)
* [Wrangling](#two)
* [Visualisation](#three)


<h2> 0. Settings </h2>  <a class="anchor" id="zero"></a>

In [1]:
!pip install pycoingecko
!pip install plotly
!pip install mplfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 71 kB 1.5 MB/s 


In [2]:
import pandas as pd
#import numpy as np
import plotly.graph_objects as go
from plotly.offline import plot
import matplotlib.pyplot as plt
import datetime
from pycoingecko import CoinGeckoAPI
from mplfinance.original_flavor import candlestick2_ohlc

<h2> 1. Data collection </h2> <a class="anchor" id="one"></a>

Let's collect the data we need !

Using the <code>get_coin_market_chart_by_id(id, vs_currency, days)</code>. <code>id</code> is the name of the coin you want, <code>vs_currency</code> is the currency you want the price in, and <code>days</code> is how many days back from today you want.

In [3]:
cg = CoinGeckoAPI()

bitcoin_data = cg.get_coin_market_chart_by_id(id='bitcoin', vs_currency='usd', days=90)

bitcoin_data is a JSON, and includes the price, market caps, and total volumes along with timestamps. 

We focus on the prices:

In [4]:
bitcoin_data_prices = bitcoin_data['prices']
print(bitcoin_data_prices[0:10])

[[1661958086501, 20163.5375347747], [1661961655724, 20132.098544993558], [1661965376537, 20080.277406943995], [1661968932900, 19984.336892382864], [1661972763249, 20016.774297299347], [1661976086952, 20219.71755747391], [1661979614424, 20206.78876713249], [1661983279797, 20195.51908677015], [1661986931327, 20164.62687248048], [1661990547274, 20021.14385357082]]


<h2> 2. Data wrangling </h2> <a class="anchor" id="two"></a>

We convert the data into a pandas dataframe, and convert the TimeStamp into a readable datetime.

In [5]:
data = pd.DataFrame(bitcoin_data_prices, columns=['TimeStamp', 'Price'])
data.head()

,TimeStamp,Price
0,1661958086501,20163.537535
1,1661961655724,20132.098545
2,1661965376537,20080.277407
3,1661968932900,19984.336892
4,1661972763249,20016.774297


In [6]:
data['date'] = data['TimeStamp'].apply(lambda d: datetime.date.fromtimestamp(d/1000))
print(data.date)

0       2022-08-31
1       2022-08-31
2       2022-08-31
3       2022-08-31
4       2022-08-31
           ...    
2150    2022-11-29
2151    2022-11-29
2152    2022-11-29
2153    2022-11-29
2154    2022-11-29
Name: date, Length: 2155, dtype: object


In [7]:
candlestick_data = data.groupby(data.date, as_index=False).agg({"Price": ['min', 'max', 'first', 'last']})
print(candlestick_data.head(5))

         date         Price                                          
                        min           max         first          last
0  2022-08-31  19984.336892  20219.717557  20163.537535  20164.626872
1  2022-09-01  19632.460382  20121.599757  20021.143854  20081.735752
2  2022-09-02  19825.516782  20385.757561  20141.167412  19962.185407
3  2022-09-03  19748.722216  20017.062558  19963.427145  19819.666689
4  2022-09-04  19704.095425  19920.195380  19814.564245  19838.028380


<h2> 3. Data visualisation </h2> <a class="anchor" id="three"></a>

Let's make a first candlestick graph:

In [8]:
fig = go.Figure(data=[go.Candlestick(x=candlestick_data['date'],
                open=candlestick_data['Price']['first'], 
                high=candlestick_data['Price']['max'],
                low=candlestick_data['Price']['min'], 
                close=candlestick_data['Price']['last'])])

fig.show()

We will improve it a bit: get rid of the range slider, add a title, an axis title, add a 7-day moving average....

In [9]:
candlestick_data['7dma'] = candlestick_data['Price']['last'].rolling(window = 7,center = True).mean()

fig.update_layout(xaxis_rangeslider_visible=False, 
template = "simple_white", 
title = "Evolution of bitcoin price (in USD) in the last 90 days", 
yaxis_title = "USD",
legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="center",
    x=0.5
))

fig.data[0].name="Candlesticks"

fig.add_trace(go.Scatter(x = candlestick_data['date'], 
    y = candlestick_data['7dma'],
    line = dict(color = "#000000"),
    name = "7-day moving average"))
fig.show()

Et voilà :) ! 

We learn to retrieve data using CoinGecko API, to prepare and aggregate the data using pandas, and to visualise it using plotly. 